<a href="https://colab.research.google.com/github/sujathasivaraman/mlai/blob/main/cnn_chicken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#@title IGNORE Google images download code
!pip install google-images-download
from google_images_download import google_images_download

def download_thumbnails(query, limit=10, output_dir='chickens\downloads'):
    response = google_images_download.googleimagesdownload()
    arguments = {
        "keywords": query,
        "limit": limit,
        "print_urls": True,
        "size": "medium",  # Ensures thumbnails or smaller images
        "output_directory": output_dir,
        "format": "jpg"
    }
    try:
        response.download(arguments)
        print(f"Thumbnails for '{query}' downloaded successfully!")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
download_thumbnails("chickens", limit=1)


# ext = ext if ext in ['jpg', 'jpeg', 'png', 'webp'] else 'jpg'
#         # Extract and clean file extension
#         ext_match = re.search(r'\.(jpg|jpeg|png|webp)', url, re.IGNORECASE)
#         ext = ext_match.group(1).lower() if ext_match else 'jpg'



Item no.: 1 --> Item name = chickens
Evaluating...
Starting Download...

Errors: 0

Thumbnails for 'chickens' downloaded successfully!


In [35]:
#@title RUN in GPU mode
# {"display-mode":"form", "form-width":"25%"}
#@markdown ##**BEFORE RUNNING ANY CODE, please change your Hardware Accelerator to GPU to train faster!**</h2>
#@markdown 1. Click on the **Runtime** menu at the top of the screen.
#@markdown 2. Click **Change Runtime Type**.
#@markdown 3. Choose **T4 GPU** under **Hardware Accelerator**.

#@markdown Once you've done that, run this code cell to check you're correctly connected!

import tensorflow as tf
from IPython.display import Markdown

if tf.test.gpu_device_name():
  display(Markdown("###✅ GPU connected!"))
else:
  display(Markdown("""
###❌ No GPU found!
If you're running into GPU limits when you try to switch, here are some suggestions:
  - Wait 12-24 hours for the limits to reset.
  - Share your copy of the notebook with another Google account that hasn't met the limit, and work through the notebook with that account.
  - Look into a paid subscription or paying for compute units as you go.
  """))


###❌ No GPU found!
If you're running into GPU limits when you try to switch, here are some suggestions:
  - Wait 12-24 hours for the limits to reset.
  - Share your copy of the notebook with another Google account that hasn't met the limit, and work through the notebook with that account.
  - Look into a paid subscription or paying for compute units as you go.
  

In [36]:
#@title  Local Drive :  Run this to load images and imports! {"display-mode":"form", "form-width":"25%"}
import tensorflow as tf
import os
import zipfile
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np  # Added for numerical operations

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Constants
IMG_SHAPE = (150, 150)  # Tuple format
BATCH_SIZE = 32  # More manageable batch size
zip_path = '/content/drive/My Drive/Dataset/chicken_notachicken.zip'
# extract_dir = '/content/drive/My Drive/Dataset/cats_and_dogs_filtered_extracted'
# zip_path = '/content/drive/My Drive/Dataset/cats_and_dogs_filtered.zip'
extract_dir = '/content/drive/My Drive/Dataset'

data_dir = '/content/drive/My drive/Dataset'
# img_size = (150,150)
# batch_size = 32

# train_generator = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=IMG_SHAPE,
#     batch_size=BATCH_SIZE,
#     class_mode='binary',
#     shuffle=True
# )

# train_dataset = image_dataset_from_directory(
#     data_dir,
#     validation_split=0.2,
#     subset="training",
#     seed=123,
#     image_size = IMG_SHAPE,
#     batch_size= BATCH_SIZE
# )

# validation_dataset = image_dataset_from_directory(
#     data_dir,
#     validation_split=0.2,
#     subset="validation",
#     seed=123,
#     image_size=img_size,
#     batch_size=batch_size
# )
#train_generator = train_datagen.flow_from_directory(
train_generator = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    image_size=IMG_SHAPE,
    seed = '123',
    batch_size=BATCH_SIZE,
    interpolation ='bilinear',
    shuffle=True
)

validation_generator = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed = '123',
    image_size=IMG_SHAPE,
    batch_size=BATCH_SIZE,
#    class_mode='binary',
    interpolation ='bilinear',
    shuffle=False
)

# # Verify class indices
# print("Class indices:", train_generator.class_indices)

# # Optional: Load sample batch for inspection
# sample_images, sample_labels = next(train_generator)
# print("Sample batch shape:", sample_images.shape)
# print("Sample labels shape:", sample_labels.shape)

# Visualize sample images
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.title(f"Label: {sample_labels[i]}")
    plt.axis('off')
plt.show()

Mounted at /content/drive


NotFoundError: Could not find directory /content/drive/My drive/Dataset

In [ ]:
#@title Configure dataset for performance
AUTOTUNE=tf.data.AUTOTUNE
train_dataset=train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset=validation_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
#@title Load pretrained model
pretrained_model = tf.kera.applications.MobileNetV2(input_shape=(150,150,3),include_top=False, weights='imagenet')

In [ ]:
#@title freeze the convuolutional Base
pretrained_model.trainable= False


In [ ]:
#@title Add a classification head
model = models.Sequential([pretrained_model, layers.GlobalAveragePooling2D(),layers.Dense(1)])

In [ ]:
#@title Complile the model
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001,loss=tf.keras.losses.binary_crossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
#@title Train the model
history=model.fit(train_dataset,epochs=10,validation_data=validation_dataset)

In [ ]:
#@title Validate the model
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']


plt.figure(figsize=(8,8))
plt.subplot(2,1,1)
plt.plot(acc,label='Training Accuracy')
plt.plot(val_acc,label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')



plt.subplot(2,1,2)
plt.plot(acc,label='Training Loss')
plt.plot(val_acc,label='Validation Loss')
plt.legend(loc='Upper right')
plt.ylabel('Cross Entrophy')
plt.ylim(min[0,1.0])
plt.title('Training and Validation loss')
plt.xlabel('epoch')
plt.show()



In [ ]:
#@title Preprocess the Image
import cv2
from tensorflow.keras.preprocessing import image

def load_and_preprocess_image(img_path):
    # Load the image file, resizing it to the input size of our model
    img = image.load_img(img_path, target_size=img_size)

    # Convert the image to array
    img_array = image.img_to_array(img)

    # Normalize the image pixels
    img_array /= 255.0

    # Expand dimensions to fit the batch size
    img_array = np.expand_dims(img_array, axis=0)

    return img_array

In [ ]:
#@title Make predictions
def predict_image(model, img_path):
    # Preprocess the image
    img_array = load_and_preprocess_image(img_path)

    # Make predictions
    predictions = model.predict(img_array)

    # Decode the predictions to the original labels
    predicted_label = "chicken" if predictions[0] > 0 else "notachicken"

    return predicted_label

In [ ]:
@title Test with real picture
img_path = "path_to_your_photo"  # Replace with the path of your image
predicted_label = predict_image(model, img_path)

print(f"The model predicts that the image is a {predicted_label}")

In [ ]:
#@title IGNORE Load data,  Display pictures
#@title Load our data
data_raw, labels_raw = load_data()
data = data_raw.astype(float)
labels = categorical_to_numpy(labels_raw)
X_train, X_test, y_train, y_test = model_selection.train_test_split(data, labels, test_size=0.2, random_state=1)

### WRITE YOUR CODE BELOW: Print the image data and its shape using the shape attribute!
#plot_one_image(data_raw, labels_raw, 300) # Play around with the number!
def show_image(image_data, label, index):
    image = image_data[index].reshape(32, 32, 3)  # assuming CIFAR-style 32x32 RGB
    plt.imshow(image)
    plt.title(f"Label: {label[index]}")
    plt.axis('off')

# Plot 10 images in a 2x5 grid
plt.figure(figsize=(12, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    show_image(data_raw, labels_raw, i)
plt.tight_layout()
plt.show()

In [ ]:
#@title Display data or images
index = np.random.randint(len(cd_train_inputs))
plt.imshow(cd_train_inputs[index] / 255)
plt.show()
print("Label:", cd_train_labels[index])

# Optional: Load sample batch for inspection
sample_images, sample_labels = next(train_generator)
print("Sample batch shape:", sample_images.shape)
print("Sample labels shape:", sample_labels.shape)

# Visualize sample images
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.title(f"Label: {sample_labels[i]}")
    plt.axis('off')
plt.show()